# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.24.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'breast-cancer-run'
project_folder = './capstone-project'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
breast-cancer-run,quick-starts-ws-141384,Link to Azure Machine Learning studio,Link to Documentation


In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "azurend-project"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 5)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded......

In [4]:
found = False
key = "breast-cancer"
description_text = "Wisconsin Breast Cancer Dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

In [ ]:
df = dataset.to_pandas_dataframe()
df.head(10)

In [ ]:
df.describe()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="diagnosis",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )


In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)
remote_run.wait_for_completion(show_output=True)

Running on remote.
No run_configuration provided, running on azurend-project with default configuration
Running on remote compute: azurend-project
Parent Run ID: AutoML_7d9aec74-330e-4112-b25c-42a09ee2a3d2

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.
         6   StandardScalerWrapper RandomForest             0:00:45       0.9888    0.9938
         2   MinMaxScaler RandomForest                      0:03:05       0.9875    0.9938
         4   MinMaxScaler RandomForest                      0:03:13       0.9915    0.9938
         7   MinMaxScaler ExtremeRandomTrees                0:00:47       0.9876    0.9938
         1   MaxAbsScaler XGBoostClassifier                 0:03:29       0.9944    0.9944
         8   MinMaxScaler ExtremeRandomTrees                0:00:42       0.9889    0.9944
        11   RobustScaler ExtremeRandomTrees                0:00:44       0.9869    0.9944
         9   SparseNorma

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
!pip install xgboost==0.90

     |████████████████████████████████| 142.8 MB 26 kB/s /s eta 0:00:01
  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.3.3
    Uninstalling xgboost-1.3.3:
      Successfully uninstalled xgboost-1.3.3


In [13]:
best_run, best_model = remote_run.get_output()
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
breast-cancer-run,AutoML_7d9aec74-330e-4112-b25c-42a09ee2a3d2_45,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [14]:
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=None,
                                                                  enable_feature_sweeping=None,
                                                                  feature_sweeping_config=None,
                                                                  feature_sweeping_timeout=None,
                                                                  featurization_config=None,
                                                                  force_text_dnn=None,
                                                                  is_cross_validation=None,
                                                                  is_onnx_compatible=None,
                                                                  logger=None,
                                                              

In [15]:
metrics = best_run.get_metrics()
metrics['accuracy']

0.9806077694235589

In [16]:
#TODO: Save the best model
import joblib
from azureml.core.model import Model

description = "Best-fit AutoML model trained on Wisoconsin Breast Cancer dataset"

os.makedirs('outputs', exist_ok=True)
joblib.dump(best_model, filename="outputs/automl-model.pkl")
automl_model = remote_run.register_model(model_name='automl-breastcancer', description=description)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [19]:
from azureml.core.webservice import AciWebservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    description='Predict tumor diagnosis - Malignant(1) or Benign(0)',
    auth_enabled=True
)

In [ ]:
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.automl.core.shared import constants

model = Model(ws, 'automl-breastcancer')


myenv = best_run.get_environment()
entry_script = 'predict.py'
#best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)
#best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
inference_config = InferenceConfig(entry_script=entry_script, environment=myenv)

service = Model.deploy(workspace=ws, 
                       name='automl-breastcancer', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aci_config)

service.wait_for_deployment(show_output=True)

In [ ]:
service.update(enable_app_insights=True)

In [ ]:
print("State "+ service.state)
print("Key " + service.get_keys()[0])
print("Swagger URI " + service.swagger_uri)
print("Scoring URI " + service.scoring_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# Model endpoint is consumed using a script which contains 2 datapoints
# Expected result [0, 1]
%run endpoint.py

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())